In [1]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
from pathlib import Path

In [2]:
def get_sentence(dataset):
    for sample in dataset:
        yield sample

In [27]:
def get_or_build_tokenizer(dataset):
    tokenizer_path = Path("tokenizer.json")
    if not Path.exists(tokenizer_path):
        tokenizer = Tokenizer(WordLevel(unk_token='<unk>'))
        tokenizer.pre_tokenizer = Whitespace()
        trainer = WordLevelTrainer(special_tokens=['<unk>'], min_frequency=1)
        tokenizer.train_from_iterator(get_sentence(dataset), trainer=trainer)
        tokenizer.save(str(tokenizer_path))
        print("Word Tokenizer Created and Saved")
    else:
        print(f"Loading tokenizer {str(tokenizer_path)}")
        tokenizer = Tokenizer.from_file(str(tokenizer_path))
    return tokenizer

In [28]:
dataset = ["प्रत्येक व्यक्तिलाई शिक्षाको अधिकार छ । कमसेकम प्रारम्भिक र आधारभूत शिक्षा निःशुल्क हुनेछ । प्रारम्भिक शिक्षा अनिवार्य हुनेछ । प्रविधिक र व्यवसायिक शिक्षा सबै सुलभ हुनेछ र उच्च शिक्षामा योग्यताको आधारमा सबैको समान पहुँच हुनेछ ।",
"शिक्षा मानव व्यक्तित्वको पूर्ण विकास र मानवअधिकार तथा मौलिक स्वतन्त्रताको सम्मानको सुद्दढिकरणतर्फनिर्देशित हुनेछ । यसले सबै राष्ट्र, जाति र धार्मिक समूहहरुबीच समझदारी, सहिष्णुता र मित्रताको प्रवर्द्धन गर्नेछ र संयुक्त राष्ट्र संघको शान्ति कायम गर्ने प्रयत्नहरुलाई अघि बढाउने छ ।"]
print(dataset[0])
print(dataset[1])

प्रत्येक व्यक्तिलाई शिक्षाको अधिकार छ । कमसेकम प्रारम्भिक र आधारभूत शिक्षा निःशुल्क हुनेछ । प्रारम्भिक शिक्षा अनिवार्य हुनेछ । प्रविधिक र व्यवसायिक शिक्षा सबै सुलभ हुनेछ र उच्च शिक्षामा योग्यताको आधारमा सबैको समान पहुँच हुनेछ ।
शिक्षा मानव व्यक्तित्वको पूर्ण विकास र मानवअधिकार तथा मौलिक स्वतन्त्रताको सम्मानको सुद्दढिकरणतर्फनिर्देशित हुनेछ । यसले सबै राष्ट्र, जाति र धार्मिक समूहहरुबीच समझदारी, सहिष्णुता र मित्रताको प्रवर्द्धन गर्नेछ र संयुक्त राष्ट्र संघको शान्ति कायम गर्ने प्रयत्नहरुलाई अघि बढाउने छ ।


In [29]:
tokenizer = get_or_build_tokenizer(dataset)

Word Tokenizer Created and Saved


In [34]:
list(tokenizer.get_vocab().items())[:2]

[('गर्ने', 18), ('धार्मिक', 22)]

In [35]:
tokenizer.get_vocab_size()

55

In [37]:
from tokenizers import Tokenizer, trainers, models, pre_tokenizers, decoders
from pathlib import Path

def get_or_build_bpe_tokenizer(dataset):
    tokenizer_path = Path("byte_level_tokenizer.json")
    
    if not Path.exists(tokenizer_path):
        tokenizer = Tokenizer(models.BPE())
        tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()
        trainer = trainers.BpeTrainer(special_tokens=["<unk>"], min_frequency=1)
        tokenizer.train_from_iterator(get_sentence(dataset), trainer=trainer)
        tokenizer.save(str(tokenizer_path))
        print("Byte-Level Tokenizer Created and Saved")
    else:
        print(f"Loading tokenizer {str(tokenizer_path)}")
        tokenizer = Tokenizer.from_file(str(tokenizer_path))

    return tokenizer

In [38]:
tokenizer = get_or_build_bpe_tokenizer(dataset)




Byte-Level Tokenizer Created and Saved


In [40]:
list(tokenizer.get_vocab().items())[:2]

[('à¤Ĥ', 93), ('à¤¨à¤µà¤ħà¤§', 161)]

In [41]:
tokenizer.get_vocab_size()

165